In [ ]:
import eumdac
import geopandas as gpd
from datetime import datetime, timedelta
import os
import shutil
from shapely.geometry import Polygon, mapping


In [ ]:
import numpy as np
from configparser import ConfigParser
from mpop import CONFIG_PATH
import os
import yaml

from netCDF4 import Dataset

In [ ]:
def load_config(CONFIG_PATH):
    with open(CONFIG_PATH) as file:
        config = yaml.safe_load(file)
        return config

import os

CONFIG_PATH = r"./config.yaml"


config = load_config(CONFIG_PATH)

# Insert your personal key and secret into the single quotes
consumer_key = config['DEFAULT']['key']
consumer_secret = config['DEFAULT']['secret']

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

print(f"This token '{token}' expires {token.expiration}")

In [ ]:
datastore = eumdac.DataStore(token)
selected_collection = datastore.get_collection('EO:EUM:DAT:METOP:SOMO25')


In [ ]:
from IPython.core.display import HTML 
# Display the details for the selected collection
display(HTML('<b>'+selected_collection.title+'</b>'))
display(HTML('<b>ID:</b> '+str(selected_collection)))
display(HTML('<b>Abstract:</b> '+selected_collection.abstract))

In [ ]:
collectionID ='EO:EUM:DAT:MSG:HRSEVIRI'

datastore = eumdac.DataStore(token)
datastore.collections

selected_collection = datastore.get_collection(collectionID)
bbox =[32.9, 3.2, 48, 15]
#products = selected_collection.search(bbox=bbox) 

In [ ]:
# Add vertices for polygon, wrapping back to the start point.
geometry = [[15,32.8],[2.9,32.8],[2.9,48],[48,15], [15,32.8]]

download_dir = r'C:\Users\Riccardo\Desktop\PhD docs\Drought prediction\dataset\EUMETSAT\MSG'

start_date = '2009-01-01 11:45:00'
end_date= '2020-01-01 12:20:00'

start_dt = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_dt = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
delta = timedelta(days=1)
time_window = timedelta(minutes=30)

while start_dt <= end_dt:  
    limit_dt = start_dt + time_window
    # Retrieve datasets that match our filter
    product = selected_collection.search(
        geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
        #bbox=bbox,
        dtstart=start_dt, 
        dtend=limit_dt).first()
        
    selected_product = datastore.get_product(product_id=str(product), collection_id=collectionID)
    
    try:
        with selected_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
            #print(f'Downloading {fsrc.name}')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {fsrc.name} finished.')
        
    except Exception as e:
        print('http error {} on day'.format(e), datetime.strftime(start_dt, format='%Y-%m-%d %H:%M:%S'))
    
    start_dt += delta


2011-03-11

In [ ]:
path = r'C:\Users\Riccardo\Desktop\PhD docs\Drought prediction\ETH_adm\gadm40_ETH_0.shp'
eth_poly = gpd.read_file(path)


### Processing files

In [ ]:
from satpy import DataQuery
from pyresample.geometry import SwathDefinition
from satpy import Scene
import xarray as xr
import geopandas as gpd
from satpy.dataset import combine_metadata
import matplotlib.pyplot as plt
from glob import glob
import os
import datetime as datetime

In [ ]:
from satpy import Scene

base_dir = r'D:\MSG\MSG_nat\batch_2'
file = r'MSG2-SEVI-MSG15-0100-NA-20110311121242.259000000Z-NA.nat'
scn =Scene(filenames = {reader:[os.path.join(base_dir,file)]})

In [ ]:
from epct import api

#fine the configuration of the functional chain to apply:
chain_config = {
       'name': 'sample_chain',
       'product': 'HRSEVIRI',
       'format': 'netcdf4',
       'projection': 'geographic',
   }

base_dir = r'D:\MSG\MSG_nat\batch_2'

files = [f for f in os.listdir(base_dir) if f.endswith('.nat')]

with open(r'C:\Users\Riccardo\Desktop\PhD_docs\Drought_prediction\ETH_adm.zip', 'rb') as f:
     shapefile_stream = f.read()

target_dir = r'D:\MSG\msg_data\batch_2'
#n the chain and return the result as an `xarray` object
output_xarray_dataset = api.run_chain_to_xarray(
   product_paths=[r'D:\\MSG\MSG_nat\batch_2\MSG2-SEVI-MSG15-0100-NA-20110311121242.259000000Z-NA.nat'],
   chain_config=chain_config,
   target_dir=target_dir,
   shapefile_stream=shapefile_stream
)

In [ ]:
lon, lat = scn[0.8].attrs['area'].get_lonlats()
swath_def = SwathDefinition(lons=lon, lats=lat)

In [ ]:
def compute_ndvi(scn):
    #lon, lat = scn[0.8].attrs['area'].get_lonlats()
    ndvi = (scn[0.8] - scn[0.6]) / (scn[0.8] + scn[0.6])
    ndvi.attrs = combine_metadata(scn[0.8], scn[0.6])
    scn['ndvi'] = ndvi
    return scn

def from_nat_netcdf(file, reader = "seviri_l1b_native"):
    scn =Scene(filenames = {reader:[file]})
    #dataset_names = scn.all_dataset_names()
    my_channel_id = DataQuery(name=['VIS006'], calibration='reflectance')
    my_channel_id_2 = DataQuery(name=['VIS008'], calibration='reflectance')
    scn.load([my_channel_id, my_channel_id_2])
    scn_ndvi = compute_ndvi(scn)
    return scn_ndvi

### Cut Ethiopia and convert to xarray

In [ ]:
import geopandas as gpd
path = r'C:\Users\Riccardo\Desktop\PhD_docs\Drought_prediction\ETH_adm\gadm40_ETH_0.shp'
eth_poly = gpd.read_file(path)


In [ ]:
from epct import api

# define the configuration of the functional chain to apply:
chain_config = {
       'name': 'sample_chain',
       'product': 'HRSEVIRI',
       'format': 'netcdf4',
       'projection': 'geographic'}

#ecify the input product paths and load the shapefile
input_products = [
    r'D:\MSG\MSG_nat\batch_2\msg2_sevi.nat',
    ]

with open(r'C:\Users\Riccardo\Downloads\gadm40_ETH_shp.zip', 'rb') as f:
    shapefile_stream = f.read()


#n the chain and return the result as an `xarray` object
output_xarray_dataset = api.run_chain_to_xarray(
   product_paths=input_products,
   chain_config=chain_config,
   target_dir=r'D:\MSG\msg_data\batch_2',
   shapefile_stream=shapefile_stream
)


In [ ]:
os.path.join(base_dir,file)

In [ ]:
from epct import api
import os
import xarray as xr

chain_config = {"filter": "hrseviri_natural_color",
        "name": "Natural color disc",
        "id": "natural_color_disc",
        'product': 'HRSEVIRI',
        'format': 'netcdf4',
        'projection': 'geographic'
    }

#input_products = [
#     r'D:\MSG\msg_data\MSG2-SEVI-MSG15-0100-NA-20090108121240.961000000Z-NA.nat'
# ]

#files = glob('D:\MSG\msg_data\*.nat')
base_dir = r'D:\MSG\MSG_nat\batch_1'
files = [f for f in os.listdir(base_dir) if f.endswith('.nat')]

with open(r'C:\Users\Riccardo\Desktop\PhD_docs\Drought_prediction\gadm40_ETH_shp.zip', 'rb') as f:
     shapefile_stream = f.read()

for file in files:
    # run the chain and return the result as an `xarray` object
    output_xarray_dataset = api.run_chain_to_xarray(
       product_paths=[os.path.join(base_dir,file)],
       chain_config=chain_config,
       target_dir=r'D:\MSG\msg_data\batch_2',
       shapefile_stream=shapefile_stream
    )

In [ ]:
files

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from glob import glob
import os

def add_time(xr_df):
    my_date_string = xr_df.attrs['date_time'] 
    date_xr = datetime.strptime(my_date_string, '%Y%m%d/%H:%M')
    date_xr = pd.to_datetime(date_xr)
    xr_df = xr_df.assign_coords(time=date_xr)
    xr_df = xr_df.expand_dims(dim="time")
    return xr_df

def compute_radiance(xr_df):
    satellite = xr_df.attrs['EPCT_product_name'][:4]
    if satellite == 'MSG2':
        xr_df['channel_1'] = xr_df['channel_1']/65.2065
        xr_df['channel_2'] = xr_df['channel_2']/73.0127
        
    elif satellite == 'MSG1':
        xr_df['channel_1'] = xr_df['channel_1']/65.2296 
        xr_df['channel_2'] = xr_df['channel_2']/73.1869
    
    else:
        print('This product doesn\'t contain MSG1 or MSG2 Seviri')
    
    return xr_df
        

base_dir = r'D:\MSG\msg_data\netcdf_data'
files = [f for f in os.listdir(base_dir) if f.endswith('.nc')]

for file in files:
    with xr.open_dataset(os.path.join(base_dir, file)) as ds:
        data = ds.load()
        xr_df = add_time(data)
        xr_df = compute_radiance(xr_df)
        xr_df = xr_df.drop('channel_3')
        xr_df = xr_df.assign(ndvi=(xr_df['channel_2'] - xr_df['channel_1']) / (xr_df['channel_2'] + xr_df['channel_1']))
        #xr_df['channel_2'].plot()
        xr_df.to_netcdf(os.path.join(base_dir,'processed', file)) 
        xr_df.close()

### Plot the average NDVI per month

In [ ]:
import sys
print(sys.prefix)

In [ ]:
import xarray as xr
base_dir = r'D:\MSG\msg_data\netcdf_data\processed'
file = 'HRSEVIRI_20090101T121200Z_20090101T121200Z_epct_c0b84031_FP.nc'
xr_df = xr.open_dataset(os.path.join(base_dir, file))

In [ ]:
xr_df['channel_1'].plot()

In [ ]:
xr_df['channel_2'].plot()

In [ ]:
xr_df['ndvi'].plot()

### Cloudmask dataset

In [ ]:
import xarray as xr
import os
base_dir = r'D:\MSG\cloudmask\processed_clouds'
file = 'MSGCLMK_20100115T121500Z_20100115T121500Z_epct_82eea5af_P.nc'
xr_df = xr.open_dataset(os.path.join(base_dir, file))
xr_df

In [ ]:
xr_df['cloud_mask'].plot()

In [ ]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt

def downsample(ds):
    monthly = ds.resample(time='5D', skipna=True).mean()
    return monthly

def clean_ndvi(ds):
    ds = ds.where('ndvi'!=0.00)
    return ds

base_dir = r'D:\MSG\msg_data\netcdf_data\processed'
ds = xr.open_mfdataset(os.path.join(base_dir,'*.nc'))
clouds_dir = r'D:\MSG\cloudmask\processed_clouds'
ds_cl = xr.open_mfdataset(os.path.join(clouds_dir,'time','*.nc'))

In [ ]:
### normalize time in order for the two datasets to match
ds_cl['time'] = ds_cl.indexes['time'].normalize()
ds['time'] = ds.indexes['time'].normalize()

### apply time mask
ds = ds.where(ds.time == ds_cl.time)

### apply mask and downsample to 5 days
res_xr = ds.where(ds_cl.cloud_mask<2)
res_xr_p = downsample(res_xr)

### mask all the values equal to 0 (clouds)
mask_clouds = clean_ndvi(ds)
mask_clouds_p = downsample(mask_clouds)

In [ ]:
time = '2009-12-05'
#print('')
ds['ndvi'].sel(time=time, method = 'nearest').plot()
plt.title('Default NDVI image no cloud correction')
plt.show()
mask_clouds_p['ndvi'].sel(time=time,  method = 'nearest').plot()
plt.title('NDVI image with max pixel value cloud correction')
plt.show()
res_xr_p['ndvi'].sel(time=time,  method = 'nearest').plot()
plt.title('NDVI image with cloud mask')
plt.show()
ds_cl['cloud_mask'].sel(time=time,  method = 'nearest').plot()
plt.title('Cloud mask')
plt.show()

### Difference in absolute value

In [ ]:
res = mask_clouds_p['ndvi'].isel(time=0) - res_xr_p['ndvi'].isel(time=0)

In [ ]:
res.plot()

In [ ]:
mask_clouds['ndvi'].sum(res_xr['ndvi'])

In [ ]:
diff = mask_clouds['ndvi'] res_xr['ndvi']
diff_p = mask_clouds_p['ndvi'] - res_xr_p['ndvi']

In [ ]:
abs(diff).mean('time', skipna=True).plot()
plt.show()

abs(diff_p).mean('time').plot()
plt.show()

### Check correlation

In [ ]:
import xskillscore as xs

# Selecting variables from the dataarray...................................
ndvi_= res_xr['ndvi'].chunk(dict(time=-1))
mask_= mask_clouds['ndvi'].chunk(dict(time=-1))

xs.pearson_r(ndvi_, mask_, dim='time', skipna=True).plot()
plt.show()

# Selecting variables from the dataarray...................................
ndvi_= res_xr_p['ndvi'].chunk(dict(time=-1))
mask_= mask_clouds_p['ndvi'].chunk(dict(time=-1))

xs.pearson_r(ndvi_, mask_, dim='time', skipna=True).plot()
plt.show()

### Calculate NDVI indices

In [ ]:
from IPython.display import Image
Image(r"C:\Users\Riccardo\Desktop\PhD docs\Drought prediction\formula_anomaly NDVI.PNG")


In [ ]:
###nomral anomaly
def lta_anomaly(ds):
    climatology = ds.groupby("time.month").mean("time")  ###12 months only, average per month
    anomalies =ds.groupby("time.month") - climatology  ### group data by month and subtract the average value
    anomalies = anomalies.drop('month')
    ds = ds.assign(ndvi_lta = anomalies['ndvi'])
    return ds

### reference anomaly with max month
def ref_anomaly(ds):
    climatology = ds.groupby("time.month").max('time').drop('crs') ###12 months only, average per month
    anomalies = ds.groupby("time.month") - climatology ### group data by month and subtract the average value
    #anomalies = anomalies.drop('month')
    ds = ds.assign(ndvi_ref = anomalies['ndvi'])
    return ds

### anomaly using daily data

def daily_anomaly(ds):
    monthly = ds.resample(time='1MS').mean()  ###resample by average value per month
    upsampled_monthly = monthly.resample(time='1D').ffill()  ### upsample filling constant values per month
    anomalies = monthly - upsampled_monthly  ### calculating the anomaly
    return anomalies

In [ ]:
#### Implementation of the formula

def lta_anomaly_(ds):
    monthly = ds['ndvi'].resample(time='1MS').mean('time')
    month_12 = ds.groupby("time.month").mean("time")
    daily = monthly.resample(time='1D').ffill()
    monthly_res = daily.resample(time='1MS').first(skipna=False)
    return (monthly - monthly_res)/ monthly_res

def ref_anomaly_(ds):
    monthly = ds['ndvi'].resample(time='1MS').mean('time')
    daily = monthly.resample(time='1D').ffill()
    monthly_res = daily.resample(time='1MS').first(skipna=False)
    return (monthly - monthly_res)/ monthly_res

In [ ]:
ds_an = lta_anomaly(res_xr)
ds_d_an = daily_anomaly(res_xr)
ds_an_ref = ref_anomaly(res_xr)

In [ ]:
ds_an.sel(time=time, method = 'nearest')['ndvi_lta'].plot()
plt.show()

ds_an_ref.sel(time=time, method = 'nearest')['ndvi_ref'].plot()
plt.show()



In [ ]:
ds_an_ref.mean('time')['ndvi_ref'].plot()

### Formulas

In [ ]:
ds_lta = lta_anomaly_(res_xr)
ds_lta.mean('time').plot()

In [ ]:
ds_ref = ref_anomaly_(res_xr)
ds_ref.mean('time').plot()

### Importing requested cloud mask data

In [ ]:
import re
import os
path = r'D:\MSG\msg_data\clouds data\extracted'
reg = r'(CFCin2009)(\d{4})1215(.*)'

r = re.compile(reg)
for root, dirs, files in os.walk(path):
    l = [os.path.join(root,x) for x in files if r.match(x)]
    
new_list = [i.rsplit('\\')[-1]for i in l]

#List all files in path
for filename in os.listdir(path):
  
    #If file is not present in list
    if filename not in new_list:
        #Get full path of file and remove it
        os.remove(os.path.join(path, filename))

In [ ]:
xr_cl = xr.open_dataset(l[0])
xr_cl['CMa'].plot()